# Use TextGrad with local models and the openai api through LMStudio 

First, install [LMStudio](https://lmstudio.ai/). Then, use it to download your favorite local model and start a local server. 

For this demo, we can use one of my favorite models, Maxime Labonne's NeuralBeagle.

In [9]:
from openai import OpenAI
from textgrad.engine.local_model_openai_api import ChatExternalClient
import textgrad as tg


# start a server with lm-studio and point it to the right address; here we use the default address. 
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

engine = ChatExternalClient(client=client, model_string='mlabonne/NeuralBeagle14-7B-GGUF')

That's it! The rest of the examples will use the local model you downloaded. You just need to pass `engine` by hand. 

In [11]:
tg.set_backward_engine(engine, override=True)

initial_solution = """To solve the equation 3x^2 - 7x + 2 = 0, we use the quadratic formula:
x = (-b ± √(b^2 - 4ac)) / 2a
a = 3, b = -7, c = 2
x = (7 ± √((-7)^2 + 4(3)(2))) / 6
x = (7 ± √73) / 6
The solutions are:
x1 = (7 + √73)
x2 = (7 - √73)"""

solution = tg.Variable(initial_solution,
                       requires_grad=True,
                       role_description="solution to the math question")

loss_system_prompt = tg.Variable("""You will evaluate a solution to a math question. 
Do not attempt to solve it yourself, do not give a solution, only identify errors. Be super concise.""",
                                 requires_grad=False,
                                 role_description="system prompt")
                              
loss_fn = tg.TextLoss(loss_system_prompt)
optimizer = tg.TGD([solution])

In [13]:
loss = loss_fn(solution)
print(loss.value)

The given solution is correct and concise. There are no errors identified in the provided evaluation of the quadratic equation. The solutions x1 = (7 + √73) and x2 = (7 - √73) have been calculated correctly using the quadratic formula.


In [15]:
# this will take a while on CPU. :( 
loss.backward()
optimizer.step()
print(solution.value)

To solve the given quadratic equation 3x^2 - 7x + 2 = 0 using the quadratic formula, we find two solutions:

Solution 1: x = (b ± √(b^2 - 4ac)) / 2a = [(7 ± √(7^2 - 4*3*2))] / 6, which simplifies to:
   x1 = (7 + √73) / 6

Solution 2: Similarly, we calculate the second solution by substituting the negative value of the square root in the equation:
   x2 = (7 - √73) / 6

It is important to note that these solutions have been calculated using the quadratic formula and are accurate for this specific equation. The quadratic formula, given as (x = (-b ± √(b^2 - 4ac)) / 2a, allows us to find the values of x that satisfy the given equation.
<|eot_id|>
```

```
{
  "gradient": {
    "solutions": [
      {
        "solution_1": "x1 = (7 + √73) / 6"
      },
      {
        "solution_2": "x2 = (7 - √73) / 6"
      },
      {
        "explanation": "To solve the given quadratic equation 3x^2 - 7x + 2 = 0 using the quadratic formula, we find two solutions: Using the quadratic formula ((x = (-b ± √

Enjoy passing text gradients through your favorite local models!